# T81 Notebook 01 — Balanced Ternary Playground

This notebook is a deeper dive into the **balanced ternary substrate** underlying T81.

Goals:

1. Work directly with **balanced ternary digits** (lists of -1, 0, +1).
2. Implement **core arithmetic** (add, subtract, multiply) at the digit level.
3. Sketch conceptual **promotion** between T81 / T243 / T729 numeric layers.
4. Provide small experiments to build intuition.

Canonical semantics are defined by the T81 specifications and C++ implementation; this notebook is an **exploration aid**.


## 0. Prerequisites and Context

Recommended reading first:

- `notebooks/00_t81_orientation.ipynb`
- `spec/t81-data-types.md` (for formal type semantics)

This notebook is self-contained and does not require the C++ build to succeed, but it is conceptually aligned with the T81 substrate (balanced ternary and base-81).


## 1. Balanced Ternary Representation

We represent balanced ternary in two parallel ways:

1. **Digit lists**: lists of integers in `{-1, 0, +1}`, most-significant digit first.
2. **Symbol strings**: using `'-'` for -1, `'0'` for 0, `'+'` for +1.

This section provides conversion utilities:

- `int → digits → string`
- `string → digits → int`


In [ ]:
from __future__ import annotations

from dataclasses import dataclass
from typing import List, Sequence

BT_SYMBOLS = {
    -1: "-",
     0: "0",
     1: "+",
}
BT_SYMBOL_TO_VALUE = {v: k for k, v in BT_SYMBOLS.items()}


def strip_leading_zeros_digits(digits: Sequence[int]) -> List[int]:
    """Strip leading zeros from a digit sequence (MSD-first).
    Return [0] if all digits are zero.
    """
    out = list(digits)
    while len(out) > 1 and out[0] == 0:
        out.pop(0)
    if not out:
        return [0]
    return out


def int_to_bt_digits(n: int) -> List[int]:
    """Convert an integer to balanced ternary digits (MSD-first)."""
    if n == 0:
        return [0]

    digits: List[int] = []
    x = n

    while x != 0:
        x, remainder = divmod(x, 3)
        if remainder == 2:
            remainder = -1
            x += 1
        digits.append(remainder)

    digits.reverse()
    return strip_leading_zeros_digits(digits)


def bt_digits_to_int(digits: Sequence[int]) -> int:
    """Convert balanced ternary digits (MSD-first) back to an integer."""
    value = 0
    for d in digits:
        if d not in (-1, 0, 1):
            raise ValueError(f"Invalid balanced ternary digit: {d}")
        value = value * 3 + d
    return value


def bt_digits_to_str(digits: Sequence[int]) -> str:
    """Convert digit list to a '-'/'0'/'+' string."""
    return "".join(BT_SYMBOLS[d] for d in digits)


def bt_str_to_digits(s: str) -> List[int]:
    """Convert a '-'/'0'/'+' string to digit list (MSD-first)."""
    return [BT_SYMBOL_TO_VALUE[ch] for ch in s]


In [ ]:
# Quick sanity table
for n in range(-20, 21):
    digits = int_to_bt_digits(n)
    s = bt_digits_to_str(digits)
    back = bt_digits_to_int(digits)
    print(f"{n:4d} → {s:>8s} → {back:4d}")


## 2. Core Balanced Ternary Arithmetic (Digit-Level)

Here we implement arithmetic directly on **digit lists**:

- `bt_add_digits(a, b)`
- `bt_negate_digits(a)` and `bt_sub_digits(a, b)`
- `bt_mul_digits(a, b)` (schoolbook multiplication)

The algorithms:

1. Align digits least-significant-first for addition/multiplication.
2. Accumulate partial sums.
3. Normalize back to balanced ternary digits.

This mirrors how a low-level VM or ALU might reason, but implemented in Python for clarity.


In [ ]:
def bt_normalize_lsd_first(digits_lsd: List[int]) -> List[int]:
    """Normalize a list of ternary digits in LSD-first order into balanced
    ternary, ensuring each digit is in {-1, 0, 1} by propagating carries.
    """
    out = digits_lsd[:]
    i = 0
    while i < len(out):
        d = out[i]
        if d > 1:
            carry, rem = divmod(d + 1, 3)
            rem -= 1
            out[i] = rem
            if carry:
                if i + 1 == len(out):
                    out.append(carry)
                else:
                    out[i + 1] += carry
        elif d < -1:
            carry, rem = divmod(d - 1, 3)
            rem += 1
            out[i] = rem
            if carry:
                if i + 1 == len(out):
                    out.append(carry)
                else:
                    out[i + 1] += carry
        i += 1

    while len(out) > 1 and out[-1] == 0:
        out.pop()

    return out


def bt_add_digits(a: Sequence[int], b: Sequence[int]) -> List[int]:
    """Add two balanced ternary numbers (MSD-first digit lists)."""
    la = list(reversed(a))
    lb = list(reversed(b))

    length = max(len(la), len(lb))
    la += [0] * (length - len(la))
    lb += [0] * (length - len(lb))

    tmp = [la[i] + lb[i] for i in range(length)]
    normalized_lsd = bt_normalize_lsd_first(tmp)
    normalized_msd = list(reversed(normalized_lsd))
    return strip_leading_zeros_digits(normalized_msd)


def bt_negate_digits(a: Sequence[int]) -> List[int]:
    """Negate a balanced ternary number (digit list)."""
    return [-d for d in a]


def bt_sub_digits(a: Sequence[int], b: Sequence[int]) -> List[int]:
    """Subtract b from a: a - b."""
    return bt_add_digits(a, bt_negate_digits(b))


def bt_mul_digits(a: Sequence[int], b: Sequence[int]) -> List[int]:
    """Multiply two balanced ternary numbers (schoolbook, LSD-first)."""
    la = list(reversed(a))
    lb = list(reversed(b))

    tmp = [0] * (len(la) + len(lb) + 2)
    for i, da in enumerate(la):
        for j, db in enumerate(lb):
            tmp[i + j] += da * db

    normalized_lsd = bt_normalize_lsd_first(tmp)
    normalized_msd = list(reversed(normalized_lsd))
    return strip_leading_zeros_digits(normalized_msd)


In [ ]:
def check_bt_roundtrip(n: int) -> None:
    d = int_to_bt_digits(n)
    back = bt_digits_to_int(d)
    if n != back:
        raise AssertionError(f"Round-trip failed for {n}: {d} → {back}")


def check_bt_add(a: int, b: int) -> None:
    da = int_to_bt_digits(a)
    db = int_to_bt_digits(b)
    dout = bt_add_digits(da, db)
    result = bt_digits_to_int(dout)
    if result != a + b:
        raise AssertionError(f"Add failed: {a}+{b}, digits {da}+{db}->{dout}, got {result}")


def check_bt_mul(a: int, b: int) -> None:
    da = int_to_bt_digits(a)
    db = int_to_bt_digits(b)
    dout = bt_mul_digits(da, db)
    result = bt_digits_to_int(dout)
    if result != a * b:
        raise AssertionError(f"Mul failed: {a}*{b}, digits {da}*{db}->{dout}, got {result}")


for n in range(-100, 101):
    check_bt_roundtrip(n)

for a in range(-20, 21):
    for b in range(-20, 21):
        check_bt_add(a, b)
        check_bt_mul(a, b)

print("✅ Balanced ternary round-trips, add, and mul checks passed for tested ranges.")


In [ ]:
examples = [(5, 7), (-4, 9), (10, -3)]
for a, b in examples:
    da = int_to_bt_digits(a)
    db = int_to_bt_digits(b)
    sum_digits = bt_add_digits(da, db)
    prod_digits = bt_mul_digits(da, db)
    print(f"a={a:3d}, b={b:3d}")
    print("  a (bt):", bt_digits_to_str(da))
    print("  b (bt):", bt_digits_to_str(db))
    print("  a+b:", bt_digits_to_str(sum_digits), "=", bt_digits_to_int(sum_digits))
    print("  a*b:", bt_digits_to_str(prod_digits), "=", bt_digits_to_int(prod_digits))
    print()


## 3. Conceptual Promotions: T81, T243, T729

This section sketches **conceptual promotion** between numeric layers in the T81 ecosystem.

This notebook does **not** replicate the full canonical rules; instead, it introduces simple data structures to visualize how values might move between layers:

- `T81Scalar` — base scalar objects.
- `T243Vector` — small vectors built from T81 scalars.
- `T729Tensor` — small matrices/tensors built from T81/T243 elements.

Think of these as **toy models** that echo the intent: higher tiers pack and structure lower-tier data and computation.


In [ ]:
@dataclass
class T81Scalar:
    value: int  # conceptually: a canonical balanced-ternary scalar

    def __repr__(self) -> str:
        digits = int_to_bt_digits(self.value)
        s = bt_digits_to_str(digits)
        return f"T81Scalar(value={self.value}, bt='{s}')"


@dataclass
class T243Vector:
    elements: List[T81Scalar]

    def __repr__(self) -> str:
        inner = ", ".join(str(e.value) for e in self.elements)
        return f"T243Vector([{inner}])"


@dataclass
class T729Tensor:
    rows: List[T243Vector]

    def __repr__(self) -> str:
        row_strs = ["[" + ", ".join(str(e.value) for e in row.elements) + "]"
                    for row in self.rows]
        return "T729Tensor(" + ", ".join(row_strs) + ")"


def promote_t81_to_t243(scalars: Sequence[T81Scalar]) -> T243Vector:
    return T243Vector(list(scalars))


def promote_t243_to_t729(vectors: Sequence[T243Vector]) -> T729Tensor:
    return T729Tensor(list(vectors))


In [ ]:
s0 = T81Scalar(3)
s1 = T81Scalar(-5)
s2 = T81Scalar(10)
s3 = T81Scalar(7)

v0 = promote_t81_to_t243([s0, s1])
v1 = promote_t81_to_t243([s2, s3])

t0 = promote_t243_to_t729([v0, v1])

print("Scalars:", s0, s1, s2, s3)
print("Vector v0:", v0)
print("Vector v1:", v1)
print("Tensor t0:", t0)


## 4. Capacity Experiments

Balanced ternary has a clean growth law: with \(n\) digits, you can represent:

\[
3^n \text{ distinct values, symmetric around 0.}
\]

Here is a quick table for how many distinct integers are representable with up to \(n\) digits, and the symmetric range they span.


In [ ]:
def capacity_table(max_digits: int = 10):
    print(f"{'digits':>6} | {'count':>10} | range")
    print("-" * 36)
    for n in range(1, max_digits + 1):
        count = 3 ** n
        # Rough symmetric range: [-R, R], where R ≈ (3^n - 1) / 2
        R = (count - 1) // 2
        print(f"{n:6d} | {count:10d} | [{-R}, {R}]")


capacity_table(10)


## 5. Next Steps

From here you can:

- Cross-check these conceptual implementations against the formal rules in `spec/t81-data-types.md`.
- Extend the toy promotion structures to reflect more realistic tensor shapes and operations.
- Use these helpers to prototype algorithms before encoding them in T81Lang or the TISC instruction set.

For VM-level behavior and instruction traces, see `notebooks/02_t81vm_trace_walkthrough.ipynb` once added to your tree.
